In [16]:
import fitz
import numpy as np
import pandas as pd
from tqdm import tqdm

import io
from PIL import Image, ImageDraw, ImageFont, ImageColor

In [17]:
vol1_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf'
vol2_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf'
vol3_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf'

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

vol1_pages = [vol1_doc[i] for i in range(vol1_doc.page_count)]
vol2_pages = [vol2_doc[i] for i in range(vol2_doc.page_count)]
vol3_pages = [vol3_doc[i] for i in range(vol3_doc.page_count)]

In [257]:
vol1_char_df = pd.read_pickle("../input/char_df/vol1_df.pkl")
vol2_char_df = pd.read_pickle("../input/char_df/vol2_df.pkl")
vol3_char_df = pd.read_pickle("../input/char_df/vol3_df.pkl")

vol1_index = list(range(616, 639)) #inclusive
vol2_index = list(range(703, 725))
vol3_index = list(range(555, 583))

In [19]:
TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

In [20]:
page_num = 616

In [258]:
def epithet_match(row):
    return row['word_num'] == 0 and \
           row['word'].isalpha() and \
           row['word'].islower()

def genus_match(row):
    return row['word_num'] == 0 and \
           row['word'].isalpha() and \
           row['word'][0].isupper() and row['word'][1:].islower()

In [259]:
vol1_char_df

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_ascender,span_descender,span_origin,span_bbox,word_num,word,char_num,char,char_origin,char_bbox
0,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,0,A,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 119.4..."
1,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,1,U,"(119.42528533935547, 136.92001342773438)","(119.42528533935547, 127.92001342773438, 128.0..."
2,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,2,T,"(127.97057342529297, 136.92001342773438)","(127.97057342529297, 127.92001342773438, 135.2..."
3,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,3,R,"(135.19717407226562, 136.92001342773438)","(135.19717407226562, 127.92001342773438, 143.1..."
4,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,4,E,"(143.0890655517578, 136.92001342773438)","(143.0890655517578, 127.92001342773438, 150.34..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764639,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,7,E,"(227.60250854492188, 299.47998046875)","(227.60250854492188, 294.22998046875, 232.2247..."
1764640,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,8,-,"(232.03839111328125, 299.47998046875)","(232.03839111328125, 294.22998046875, 234.3461..."
1764641,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,9,S,"(234.15965270996094, 299.47998046875)","(234.15965270996094, 294.22998046875, 238.0137..."
1764642,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,10,I,"(237.82630920410156, 299.47998046875)","(237.82630920410156, 294.22998046875, 240.5231..."


In [260]:
vol1_char_df[(vol1_char_df['page_num'] == page_num)& (vol1_char_df[(vol1_char_df['page_num'] == page_num)].apply(genus_match, axis = 1))].iloc[:,:-4].drop_duplicates()

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,span_size,span_flags,span_font,span_color,span_ascender,span_descender,span_origin,span_bbox,word_num,word
1620109,1,581,1,6,"(54.959999084472656, 45.697425842285156, 417.8...",0,0,"(1.0, 0.0)","(54.959999084472656, 45.697425842285156, 417.8...",0,10.658931,20,Times-Bold,0,0.942,-0.211,"(54.959999084472656, 55.4000244140625)","(54.959999084472656, 45.697425842285156, 84.89...",0,Silène
1620759,1,581,7,21,"(32.880001068115234, 163.8463897705078, 417.38...",0,0,"(1.0, 0.0)","(32.880001068115234, 163.8463897705078, 416.99...",0,9.825574,4,Times-Roman,0,0.912,-0.211,"(32.880001068115234, 173.239990234375)","(32.880001068115234, 163.8463897705078, 67.961...",0,Berkacha
1621117,1,581,10,27,"(32.63999938964844, 205.3664093017578, 417.377...",1,0,"(1.0, 0.0)","(32.63999938964844, 214.9663848876953, 107.779...",0,9.825574,4,Times-Roman,0,0.912,-0.211,"(32.63999938964844, 224.3599853515625)","(32.63999938964844, 214.9663848876953, 107.779...",0,Tell
1621133,1,581,11,28,"(50.400001525878906, 227.68638610839844, 158.4...",0,0,"(1.0, 0.0)","(50.400001525878906, 227.68638610839844, 158.4...",0,9.825574,4,Times-Roman,0,0.912,-0.211,"(50.400001525878906, 237.07998657226562)","(50.400001525878906, 227.68638610839844, 158.4...",0,Aire
1621154,1,581,12,32,"(54.720001220703125, 246.8173828125, 417.74990...",0,0,"(1.0, 0.0)","(54.720001220703125, 246.8173828125, 417.74990...",0,10.658931,20,Times-Bold,0,0.942,-0.211,"(54.720001220703125, 256.5199890136719)","(54.720001220703125, 246.8173828125, 84.652580...",0,Silène
1621281,1,581,14,35,"(32.63999938964844, 270.1663818359375, 417.811...",0,0,"(1.0, 0.0)","(32.63999938964844, 270.1663818359375, 417.811...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(32.63999938964844, 279.55999755859375)","(32.63999938964844, 270.1663818359375, 350.988...",0,Tiges
1621718,1,581,16,38,"(54.47999954223633, 343.77740478515625, 417.46...",0,0,"(1.0, 0.0)","(54.47999954223633, 343.77740478515625, 417.46...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(54.47999954223633, 353.4800109863281)","(54.47999954223633, 344.08642578125, 85.598159...",0,Forma
1621927,1,581,19,44,"(32.63999938964844, 391.84637451171875, 417.67...",0,0,"(1.0, 0.0)","(32.63999938964844, 391.84637451171875, 417.67...",0,9.825574,4,Times-Roman,0,0.912,-0.211,"(32.63999938964844, 401.239990234375)","(32.63999938964844, 391.84637451171875, 118.15...",0,Kneissé
1622079,1,581,21,48,"(50.15999984741211, 412.96636962890625, 365.77...",1,0,"(1.0, 0.0)","(50.15999984741211, 425.9263916015625, 202.339...",0,9.825574,4,Times-Roman,0,0.912,-0.211,"(50.15999984741211, 435.32000732421875)","(50.15999984741211, 425.9263916015625, 202.339...",0,Aire
1622112,1,581,22,52,"(54.0, 445.0574035644531, 417.7076416015625, 4...",0,0,"(1.0, 0.0)","(54.0, 445.0574035644531, 417.7076416015625, 4...",0,10.658931,20,Times-Bold,0,0.942,-0.211,"(54.0, 454.760009765625)","(54.0, 445.0574035644531, 84.13069915771484, 4...",0,Silène


In [261]:
vol1_char_df[(vol1_char_df['page_num'] == page_num)& (vol1_char_df[(vol1_char_df['page_num'] == page_num)].apply(epithet_match, axis = 1))].iloc[:,:-4].drop_duplicates()

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,span_size,span_flags,span_font,span_color,span_ascender,span_descender,span_origin,span_bbox,word_num,word
1620303,1,581,3,9,"(33.36000061035156, 81.28639221191406, 417.716...",0,0,"(1.0, 0.0)","(33.36000061035156, 81.28639221191406, 417.716...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(33.36000061035156, 90.67999267578125)","(33.36000061035156, 81.28639221191406, 315.128...",0,celles
1620591,1,581,4,12,"(32.880001068115234, 104.80641174316406, 418.3...",2,0,"(1.0, 0.0)","(32.880001068115234, 128.32640075683594, 380.9...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(32.880001068115234, 137.72000122070312)","(32.880001068115234, 128.32640075683594, 380.9...",0,réduite
1621210,1,581,13,33,"(33.119998931884766, 258.64642333984375, 418.0...",0,0,"(1.0, 0.0)","(33.119998931884766, 258.64642333984375, 418.0...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(33.119998931884766, 268.0400085449219)","(33.119998931884766, 258.64642333984375, 418.0...",0,à
1621504,1,581,14,35,"(32.63999938964844, 270.1663818359375, 417.811...",3,0,"(1.0, 0.0)","(32.63999938964844, 304.72637939453125, 417.32...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(32.63999938964844, 314.1199951171875)","(32.63999938964844, 304.72637939453125, 417.32...",0,court
1621781,1,581,17,40,"(32.63999938964844, 356.806396484375, 318.8301...",0,0,"(1.0, 0.0)","(32.63999938964844, 356.806396484375, 318.8301...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(32.63999938964844, 366.20001220703125)","(32.63999938964844, 356.806396484375, 85.37083...",0,sommet
1622168,1,581,23,53,"(32.15999984741211, 457.12640380859375, 417.81...",0,0,"(1.0, 0.0)","(32.400001525878906, 457.12640380859375, 417.5...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(32.400001525878906, 466.52001953125)","(32.400001525878906, 457.12640380859375, 417.5...",0,couverte
1622239,1,581,23,53,"(32.15999984741211, 457.12640380859375, 417.81...",1,0,"(1.0, 0.0)","(32.15999984741211, 468.88641357421875, 417.56...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(32.15999984741211, 478.2799987792969)","(32.15999984741211, 468.88641357421875, 417.56...",0,stériles
1622317,1,581,23,53,"(32.15999984741211, 457.12640380859375, 417.81...",2,0,"(1.0, 0.0)","(32.400001525878906, 480.6463623046875, 417.81...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(32.400001525878906, 490.03997802734375)","(32.400001525878906, 480.6463623046875, 417.81...",0,ainsi
1622467,1,581,24,55,"(32.15999984741211, 492.4063720703125, 417.709...",1,0,"(1.0, 0.0)","(32.400001525878906, 504.1663818359375, 417.67...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(32.400001525878906, 513.5599975585938)","(32.400001525878906, 504.1663818359375, 417.67...",0,à
1622539,1,581,24,55,"(32.15999984741211, 492.4063720703125, 417.709...",2,0,"(1.0, 0.0)","(32.15999984741211, 515.9263916015625, 417.709...",0,10.402490,4,Times-Roman,0,0.912,-0.211,"(32.15999984741211, 525.3200073242188)","(32.15999984741211, 515.9263916015625, 417.709...",0,pétales


In [262]:
#rightmost point of any bounding box:
def get_center_x0_INDEXONLY(vol_char_df, page_num):
    df = vol_char_df[vol_char_df['page_num'] == page_num]
    
    right_bound = df['char_bbox'].apply(lambda x : x[2]).max() 
    #leftmost point of any bounding box:
    left_bound = df['char_bbox'].apply(lambda x : x[0]).min()

    return 0.5*(right_bound + left_bound) - 30


def get_col_num(coords, center_x0):
    x0, y0, x1, y1 = coords
    return int(x0 >= center_x0)


all_vol_data = [(vol1_char_df, vol1_index, vol1_doc),
                (vol2_char_df, vol2_index, vol2_doc),
                (vol3_char_df, vol3_index, vol3_doc)]

for vol_char_df, vol_index, doc in all_vol_data: 
    #for each volume 
    
    vol_char_df['genus_index_pat_match'] = vol_char_df.apply(lambda r : r['page_num'] in vol_index and genus_match(r), axis = 1) #does this for whole books which is bad
    vol_char_df['epithet_index_pat_match'] = vol_char_df.apply(lambda r : r['page_num'] in vol_index and epithet_match(r), axis = 1) #does this for whole books which is bad
    for page_num in tqdm(vol_index):
        center_x0 = get_center_x0_INDEXONLY(vol_char_df, page_num)
        vol_char_df['col_num'] = vol_char_df['span_bbox'].apply(lambda coords : get_col_num(coords, center_x0)) 

100%|██████████| 28/28 [00:35<00:00,  1.25s/it]


In [263]:
#page_num = 616
# vol_char_df = vol3_char_df
# vol_index = vol3_index
# doc = vol3_doc

# image_list = []
# output_name = "strickt_match_vol3"

all_vol_data = [(vol1_char_df, vol1_index, vol1_doc, "strickt_match_vol1"),
                (vol2_char_df, vol2_index, vol2_doc, "strickt_match_vol2"),
                (vol3_char_df, vol3_index, vol3_doc, "strickt_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data: 
    #for each volume 
    image_list = []

    max_width_genus_char = vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()
    max_width_epithet_char = vol_char_df[(vol_char_df["epithet_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        genus_db = vol_char_df[(vol_char_df['page_num'] == page_num)
                                & (vol_char_df['genus_index_pat_match'] == True)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['epithet_index_pat_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #genus pattern match flag should check with half page and not entire page:
        for col in range(2):
            num_genus_col = genus_db[genus_db["col_num"] == col].shape[0]
            num_epithet_col = epithet_db[epithet_db["col_num"] == col].shape[0]
        if num_genus_col <= 2:
            print(f"***FLAG***\n number of patterns matched for genus \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
        if epithet_db.shape[0] <= 2:
            print(f"***FLAG***\n number of patterns matched for epithet \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")

        #genus is blue, 4
        for coord in genus_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=4)
        #epithet is red, 3
        for coord in epithet_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#990000"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

 65%|██████▌   | 15/23 [00:06<00:03,  2.64it/s]

***FLAG***
 number of patterns matched for genus 
 in page 15 col 1 is 2


 87%|████████▋ | 20/23 [00:07<00:01,  2.67it/s]

***FLAG***
 number of patterns matched for genus 
 in page 20 col 1 is 0


 18%|█▊        | 4/22 [00:01<00:07,  2.43it/s]

...

 86%|████████▌ | 24/28 [00:09<00:01,  2.72it/s]

***FLAG***
 number of patterns matched for genus 
 in page 24 col 1 is 2


 93%|█████████▎| 26/28 [00:09<00:00,  2.63it/s]

***FLAG***
 number of patterns matched for genus 
 in page 26 col 1 is 0


100%|██████████| 28/28 [00:10<00:00,  2.64it/s]


In [332]:
def is_coord_match(x, x_ref_left, x_ref_right, margin):
    return (x_ref_left - margin <= x[0] and x[0] <= x_ref_left + margin) or (x_ref_right - margin <= x[0] and x[0] <= x_ref_right + margin)

In [265]:

vol_char_df = vol1_char_df 
vol_index = vol1_index

all_vol_data = [(vol1_char_df, vol1_index),
                (vol2_char_df, vol2_index),
                (vol3_char_df, vol3_index)]

for vol_char_df, vol_index in all_vol_data: 
    vol_char_df["genus_coord_match"] = vol_char_df["span_bbox"].apply(lambda x : False)
    for page_num in tqdm(vol_index):
        margin = 1.25 * vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).mean()
        genus_char_df = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["genus_index_pat_match"] == True)]
        genus_df = genus_char_df.loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])].drop_duplicates()
        page_genus_2dic = [{}, {}]
        
        for i in range(genus_df.shape[0]):
            g_index = str(page_num) + "_" + str(i)
            p0 = genus_df['span_bbox'].iloc[i]
            col = genus_df['col_num'].iloc[i]
            x_ref = p0[0]

            ref_neighbors_df = genus_df[(genus_df["page_num"] == page_num) & 
                                        (genus_df["span_bbox"].apply(lambda x : x_ref - margin <= x[0] and x[0] <= x_ref + margin))]
            #should say span and not have char_num to be more accurate. This only accounts for errors with span changing... 
            # ref_neighbors_df = vol_char_df[(vol_char_df["page_num"] == page_num) & 
            #                                (vol_char_df["char_num"] < 2) &
            #                                (vol_char_df["char_bbox"].apply(lambda x : x_ref - margin <= x[0] and x[0] <= x_ref + margin))]
        
            num_neighbors = ref_neighbors_df.shape[0]
            mean_neighbors = ref_neighbors_df["span_bbox"].apply(lambda x : x[0]).mean()
            page_genus_2dic[col][g_index] = (num_neighbors, mean_neighbors)
        
        mean_left_genus = max(page_genus_2dic[0].values(), default = [-1, -1])[1]
        mean_right_genus = max(page_genus_2dic[1].values(), default = [-1, -1])[1]

        if mean_left_genus == -1 or mean_right_genus == -1:
            mean_valid_col = max(mean_left_genus, mean_right_genus)
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "genus_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["span_bbox"].apply(lambda x : is_coord_match(x, mean_valid_col, mean_valid_col, margin))
        elif mean_left_genus == -1 and mean_right_genus == -1:
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "genus_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["span_bbox"].apply(lambda x : False)
        else: 
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "genus_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["span_bbox"].apply(lambda x : is_coord_match(x, mean_left_genus, mean_right_genus, margin))

100%|██████████| 28/28 [00:01<00:00, 26.14it/s]


In [266]:
vol1_df = vol1_char_df.loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])].drop_duplicates()

In [267]:
vol1_df[(vol1_df["genus_coord_match"] == True) & (vol1_df["genus_index_pat_match"] == False) ]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_ascender,span_descender,span_origin,span_bbox,word_num,word,genus_index_pat_match,epithet_index_pat_match,col_num,genus_coord_match
1716560,1,616,53,86,"(234.47999572753906, 161.3843994140625, 275.75...",0,0,"(1.0, 0.0)","(234.47999572753906, 161.3843994140625, 275.75...",0,...,0.959,-0.322,"(234.47999572753906, 169.44000244140625)","(234.47999572753906, 161.3843994140625, 275.75...",0,AgropyrQn,False,False,1,True
1717131,1,616,76,126,"(232.55999755859375, 535.7843627929688, 285.89...",0,0,"(1.0, 0.0)","(232.55999755859375, 535.7843627929688, 285.89...",0,...,0.959,-0.322,"(232.55999755859375, 543.8399658203125)","(232.55999755859375, 535.7843627929688, 285.89...",0,AIZOACEAE,False,False,1,True
1717352,1,617,0,0,"(29.280000686645508, 27.321340560913086, 260.3...",0,0,"(1.0, 0.0)","(29.280000686645508, 27.321340560913086, 45.93...",0,...,0.920,-0.260,"(29.280000686645508, 37.47998046875)","(29.280000686645508, 27.321340560913086, 45.93...",0,542,False,False,0,True
1717457,1,617,5,9,"(29.520000457763672, 91.58441162109375, 209.30...",0,0,"(1.0, 0.0)","(29.520000457763672, 91.58441162109375, 103.98...",0,...,0.959,-0.322,"(29.520000457763672, 99.6400146484375)","(29.520000457763672, 91.58441162109375, 103.98...",0,ALISMATACEAE,False,False,0,True
1719498,1,617,88,158,"(237.36000061035156, 553.4992065429688, 302.30...",0,0,"(1.0, 0.0)","(237.36000061035156, 553.4992065429688, 302.30...",0,...,0.912,-0.211,"(237.36000061035156, 561.1600341796875)","(237.36000061035156, 553.4992065429688, 302.30...",0,ALSINOIDEAE,False,False,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1762617,1,638,15,27,"(221.0399932861328, 60.010009765625, 323.67431...",1,0,"(1.0, 0.0)","(221.0399932861328, 69.1300048828125, 323.6743...",0,...,0.750,-0.250,"(221.0399932861328, 75.8800048828125)","(221.0399932861328, 69.1300048828125, 275.6520...",1,arrhiza,False,False,1,True
1762660,1,638,19,31,"(219.60000610351562, 117.6099853515625, 307.97...",0,0,"(1.0, 0.0)","(219.83999633789062, 117.6099853515625, 264.15...",0,...,0.750,-0.250,"(219.83999633789062, 124.3599853515625)","(219.83999633789062, 117.6099853515625, 254.02...",1,mays,False,False,1,True
1762673,1,638,19,31,"(219.60000610351562, 117.6099853515625, 307.97...",1,0,"(1.0, 0.0)","(219.60000610351562, 126.73001098632812, 307.9...",0,...,0.750,-0.250,"(219.60000610351562, 133.48001098632812)","(219.60000610351562, 126.73001098632812, 278.6...",1,pendula,False,False,1,True
1762695,1,638,20,33,"(219.83999633789062, 136.32998657226562, 379.7...",0,0,"(1.0, 0.0)","(219.83999633789062, 136.32998657226562, 379.7...",0,...,0.750,-0.250,"(219.83999633789062, 143.07998657226562)","(219.83999633789062, 136.32998657226562, 307.9...",1,biebersteiniana,False,False,1,True


In [328]:
def longest_nonconsecutive_subset(words):
    n = len(words)
    table = [[0, []] for _ in range(n)]
    table[0][1] = [words[0]]

    for i in range(1, n):
        max_len = 0
        max_subset = []
        for j in range(i):
            if words[j] < words[i] and len(table[j][1]) > max_len:
                max_len = len(table[j][1])
                max_subset = table[j][1]
        table[i][0] = max_len
        table[i][1] = max_subset + [words[i]]

    return table[n-1][1]

In [329]:
#Attempt at making it alphabetical ... isn't working so well ... hmm 

"""#page_num = 616
# vol_char_df = vol3_char_df
# vol_index = vol3_index
# doc = vol3_doc

# image_list = []
# output_name = "strickt_match_vol3"

all_vol_data = [(vol1_char_df, vol1_index, vol1_doc, "alphabetic_coord_match_vol1"),
                (vol2_char_df, vol2_index, vol2_doc, "alphabetic_coord_match_vol2"),
                (vol3_char_df, vol3_index, vol3_doc, "alphabetic_coord_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data: 
    #for each volume 
    image_list = []

    max_width_genus_char = vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()
    max_width_epithet_char = vol_char_df[(vol_char_df["epithet_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        genus_db = vol_char_df[(vol_char_df['page_num'] == page_num)
                                & (vol_char_df['genus_index_pat_match'] == True) 
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        genus_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['genus_coord_match'] == True) & 
                                     (vol_char_df['word'].str.isupper() == False) & 
                                     (vol_char_df['word'].str.isnumeric() == False)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['epithet_index_pat_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #genus pattern match flag should check with half page and not entire page:
        for col in range(2):
            num_genus_col = genus_db[genus_db["col_num"] == col].shape[0]
            num_epithet_col = epithet_db[epithet_db["col_num"] == col].shape[0]
        if num_genus_col <= 2:
            print(f"***FLAG***\n number of patterns matched for genus \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
        if num_epithet_col <= 2:
            print(f"***FLAG***\n number of patterns matched for epithet \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
 
        #genus Coord is pink, 5
        for coord in genus_coord_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)

        #genus is blue, 3 
        # for coord in genus_db['span_bbox'] :
        #     x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
        #     draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)

        lst = longest_nonconsecutive_subset(list(genus_coord_db[genus_coord_db["word_num"] == 0]["word"].str.lower())) 
        pruned_lst = [el for el in lst if el >= "a" and el <= "zzz"]
        genus_coord_db_alphabetic = genus_coord_db[genus_coord_db['word'].str.lower().isin(pruned_lst)]

        #genus is blue, 3 
        for coord in genus_coord_db_alphabetic['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=2)
        #epithet is red, 3
        # for coord in epithet_db['span_bbox'] :
        #     x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
        #     draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#990000"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])"""

 65%|██████▌   | 15/23 [00:06<00:03,  2.23it/s]

***FLAG***
 number of patterns matched for genus 
 in page 15 col 1 is 2


 87%|████████▋ | 20/23 [00:09<00:01,  2.25it/s]

***FLAG***
 number of patterns matched for genus 
 in page 20 col 1 is 0


100%|██████████| 23/23 [00:10<00:00,  2.23it/s]

...

 86%|████████▌ | 24/28 [00:10<00:01,  2.36it/s]

***FLAG***
 number of patterns matched for genus 
 in page 24 col 1 is 2


 93%|█████████▎| 26/28 [00:11<00:00,  2.35it/s]

***FLAG***
 number of patterns matched for genus 
 in page 26 col 1 is 0


100%|██████████| 28/28 [00:11<00:00,  2.35it/s]


In [330]:
#page_num = 616
# vol_char_df = vol3_char_df
# vol_index = vol3_index
# doc = vol3_doc

# image_list = []
# output_name = "strickt_match_vol3"

all_vol_data = [(vol1_char_df, vol1_index, vol1_doc, "postAlpha_coord_match_vol1"),
                (vol2_char_df, vol2_index, vol2_doc, "postAlpha_coord_match_vol2"),
                (vol3_char_df, vol3_index, vol3_doc, "postAlpha_coord_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data: 
    #for each volume 
    image_list = []

    max_width_genus_char = vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()
    max_width_epithet_char = vol_char_df[(vol_char_df["epithet_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        genus_db = vol_char_df[(vol_char_df['page_num'] == page_num)
                                & (vol_char_df['genus_index_pat_match'] == True) 
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        genus_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['genus_coord_match'] == True) & 
                                     (vol_char_df['word'].str.isupper() == False) & 
                                     (vol_char_df['word'].str.isnumeric() == False)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['epithet_index_pat_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #genus pattern match flag should check with half page and not entire page:
        for col in range(2):
            num_genus_col = genus_db[genus_db["col_num"] == col].shape[0]
            num_epithet_col = epithet_db[epithet_db["col_num"] == col].shape[0]
        if num_genus_col <= 2:
            print(f"***FLAG***\n number of patterns matched for genus \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
        if num_epithet_col <= 2:
            print(f"***FLAG***\n number of patterns matched for epithet \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
 
        #genus Coord is orange-pinkish, 5
        for coord in genus_coord_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)

        #genus is blue, 3 
        for coord in genus_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)

        #epithet is red, 3
        for coord in epithet_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#990000"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

 65%|██████▌   | 15/23 [00:06<00:03,  2.21it/s]

***FLAG***
 number of patterns matched for genus 
 in page 15 col 1 is 2


 87%|████████▋ | 20/23 [00:09<00:01,  2.23it/s]

***FLAG***
 number of patterns matched for genus 
 in page 20 col 1 is 0


100%|██████████| 23/23 [00:10<00:00,  2.23it/s]

...

 86%|████████▌ | 24/28 [00:10<00:01,  2.40it/s]

***FLAG***
 number of patterns matched for genus 
 in page 24 col 1 is 2


 93%|█████████▎| 26/28 [00:10<00:00,  2.39it/s]

***FLAG***
 number of patterns matched for genus 
 in page 26 col 1 is 0


100%|██████████| 28/28 [00:11<00:00,  2.40it/s]


In [319]:
genus_coord_vol1_df = vol1_char_df[(vol1_char_df['genus_coord_match'] == True) & 
                             (vol1_char_df['word_num'] == 0) &
                             (vol1_char_df['word'].str.isupper() == False) & 
                             (vol1_char_df['word'].str.isnumeric() == False)
                            ].loc[:,~vol1_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

In [320]:
vol1_df

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_ascender,span_descender,span_origin,span_bbox,word_num,word,genus_index_pat_match,epithet_index_pat_match,col_num,genus_coord_match
0,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,False,False,0,False
6,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",1,OUVRAGES,False,False,0,False
14,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",2,DU,False,False,0,False
16,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",3,MÊME,False,False,0,False
20,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",4,AUTEUR,False,False,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764614,1,641,4,6,"(154.0800018310547, 284.6300048828125, 227.289...",0,0,"(1.0, 0.0)","(154.0800018310547, 284.6300048828125, 227.289...",0,...,0.75,-0.25,"(154.0800018310547, 289.8800048828125)","(154.0800018310547, 284.6300048828125, 227.289...",2,JUILLET,False,False,0,False
1764621,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",0,MIL,False,False,0,False
1764624,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",1,NEUF,False,False,0,False
1764628,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",2,CENT,False,False,0,False


In [321]:
vol1_df[(vol1_df['page_num'] == vol1_index[17]) & (vol1_df['genus_index_pat_match'] == True)]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_ascender,span_descender,span_origin,span_bbox,word_num,word,genus_index_pat_match,epithet_index_pat_match,col_num,genus_coord_match
1752041,1,633,2,4,"(36.47999954223633, 61.79998779296875, 217.725...",0,0,"(1.0, 0.0)","(36.47999954223633, 61.79998779296875, 64.2280...",0,...,0.75,-0.25,"(36.47999954223633, 67.79998779296875)","(36.47999954223633, 61.79998779296875, 62.2492...",0,Pycreus,True,False,0,True
1752699,1,633,39,63,"(35.52000045776367, 438.80999755859375, 216.32...",0,0,"(1.0, 0.0)","(35.52000045776367, 438.80999755859375, 111.45...",0,...,0.75,-0.25,"(35.52000045776367, 445.55999755859375)","(35.52000045776367, 438.80999755859375, 98.363...",0,Queria,True,False,0,True
1752778,1,633,44,69,"(35.040000915527344, 515.6099853515625, 216.08...",0,0,"(1.0, 0.0)","(35.040000915527344, 515.6099853515625, 93.895...",0,...,0.75,-0.25,"(35.040000915527344, 522.3599853515625)","(35.040000915527344, 515.6099853515625, 91.669...",0,Rhizocephalus,True,False,0,True
1752916,1,633,52,81,"(34.560001373291016, 591.9299926757812, 216.26...",0,0,"(1.0, 0.0)","(34.560001373291016, 591.9299926757812, 71.788...",0,...,0.75,-0.25,"(34.560001373291016, 598.6799926757812)","(34.560001373291016, 591.9299926757812, 69.561...",0,Rœgneria,True,False,0,True
1752956,1,633,54,84,"(243.36000061035156, 51.45001220703125, 277.49...",0,0,"(1.0, 0.0)","(243.36000061035156, 51.45001220703125, 277.49...",0,...,0.75,-0.25,"(243.36000061035156, 58.20001220703125)","(243.36000061035156, 51.45001220703125, 277.49...",0,Romulea,True,False,1,True
1753028,1,633,57,90,"(257.0400085449219, 89.8499755859375, 319.8390...",0,0,"(1.0, 0.0)","(257.0400085449219, 89.8499755859375, 319.8390...",0,...,0.75,-0.25,"(257.0400085449219, 96.5999755859375)","(257.0400085449219, 89.8499755859375, 292.2192...",0,Phoenicia,True,False,1,False
1753055,1,633,59,94,"(243.36000061035156, 109.55999755859375, 274.4...",0,0,"(1.0, 0.0)","(243.36000061035156, 109.55999755859375, 274.4...",0,...,0.75,-0.25,"(243.36000061035156, 115.55999755859375)","(243.36000061035156, 109.55999755859375, 274.4...",0,Rottbœlia,True,False,1,True
1753163,1,633,62,99,"(243.1199951171875, 175.76998901367188, 270.82...",0,0,"(1.0, 0.0)","(243.1199951171875, 175.76998901367188, 270.82...",0,...,0.75,-0.25,"(243.1199951171875, 182.51998901367188)","(243.1199951171875, 175.76998901367188, 270.82...",0,Rumex,True,False,1,True
1753527,1,633,71,116,"(255.60000610351562, 435.239990234375, 311.361...",0,0,"(1.0, 0.0)","(255.60000610351562, 435.239990234375, 311.361...",0,...,0.75,-0.25,"(255.60000610351562, 441.239990234375)","(255.60000610351562, 435.239990234375, 283.153...",0,Syriacus,True,False,1,False
1753564,1,633,72,117,"(242.16000366210938, 463.04998779296875, 270.3...",0,0,"(1.0, 0.0)","(242.16000366210938, 463.04998779296875, 270.3...",0,...,0.75,-0.25,"(242.16000366210938, 469.79998779296875)","(242.16000366210938, 463.04998779296875, 270.3...",0,Ruppia,True,False,1,True


In [322]:
vol1_df[(vol1_df['page_num'] == vol1_index[17]) & (vol1_df['genus_coord_match'] == True)]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_ascender,span_descender,span_origin,span_bbox,word_num,word,genus_index_pat_match,epithet_index_pat_match,col_num,genus_coord_match
1752006,1,633,0,0,"(37.20000076293945, 27.22132682800293, 270.115...",0,0,"(1.0, 0.0)","(37.20000076293945, 27.22132682800293, 53.2896...",0,...,0.77002,-0.22998,"(37.20000076293945, 34.91998291015625)","(37.20000076293945, 27.22132682800293, 53.2896...",0,558,False,False,0,True
1752041,1,633,2,4,"(36.47999954223633, 61.79998779296875, 217.725...",0,0,"(1.0, 0.0)","(36.47999954223633, 61.79998779296875, 64.2280...",0,...,0.75000,-0.25000,"(36.47999954223633, 67.79998779296875)","(36.47999954223633, 61.79998779296875, 62.2492...",0,Pycreus,True,False,0,True
1752067,1,633,4,8,"(36.959999084472656, 103.530029296875, 217.600...",0,0,"(1.0, 0.0)","(36.959999084472656, 103.530029296875, 70.7823...",0,...,0.75000,-0.25000,"(36.959999084472656, 110.280029296875)","(36.959999084472656, 103.530029296875, 70.7823...",0,Quercus,False,False,0,True
1752699,1,633,39,63,"(35.52000045776367, 438.80999755859375, 216.32...",0,0,"(1.0, 0.0)","(35.52000045776367, 438.80999755859375, 111.45...",0,...,0.75000,-0.25000,"(35.52000045776367, 445.55999755859375)","(35.52000045776367, 438.80999755859375, 98.363...",0,Queria,True,False,0,True
1752705,1,633,39,63,"(35.52000045776367, 438.80999755859375, 216.32...",0,0,"(1.0, 0.0)","(35.52000045776367, 438.80999755859375, 111.45...",0,...,0.75000,-0.25000,"(35.52000045776367, 445.55999755859375)","(35.52000045776367, 438.80999755859375, 98.363...",1,hispanica,False,False,0,True
1752720,1,633,40,64,"(35.279998779296875, 476.9700012207031, 216.12...",0,0,"(1.0, 0.0)","(35.279998779296875, 476.9700012207031, 113.53...",0,...,0.75000,-0.25000,"(35.279998779296875, 483.7200012207031)","(35.279998779296875, 476.9700012207031, 113.53...",0,RAFFLESIACEAE,False,False,0,True
1752737,1,633,41,65,"(35.279998779296875, 486.57000732421875, 216.3...",0,0,"(1.0, 0.0)","(35.279998779296875, 486.57000732421875, 65.30...",0,...,0.75000,-0.25000,"(35.279998779296875, 493.32000732421875)","(35.279998779296875, 486.57000732421875, 65.30...",0,Rheum,False,False,0,True
1752778,1,633,44,69,"(35.040000915527344, 515.6099853515625, 216.08...",0,0,"(1.0, 0.0)","(35.040000915527344, 515.6099853515625, 93.895...",0,...,0.75000,-0.25000,"(35.040000915527344, 522.3599853515625)","(35.040000915527344, 515.6099853515625, 91.669...",0,Rhizocephalus,True,False,0,True
1752813,1,633,46,71,"(34.79999923706055, 534.5700073242188, 215.886...",0,0,"(1.0, 0.0)","(34.79999923706055, 534.5700073242188, 92.2794...",0,...,0.75000,-0.25000,"(34.79999923706055, 541.3200073242188)","(34.79999923706055, 534.5700073242188, 92.2794...",0,Rhynchospora,False,False,0,True
1752852,1,633,49,76,"(34.79999923706055, 563.3699951171875, 216.266...",0,0,"(1.0, 0.0)","(34.79999923706055, 563.3699951171875, 77.2274...",0,...,0.75000,-0.25000,"(34.79999923706055, 570.1199951171875)","(34.79999923706055, 563.3699951171875, 77.2274...",0,Robbairea,False,False,0,True


In [323]:
lst = longest_nonconsecutive_subset(list(genus_coord_vol1_df["word"].str.lower())) 
pruned_lst = [el for el in lst if el >= "a" and el <= "zzz"]

In [324]:
len(lst)

396

In [327]:
lst

['acanthophyllum',
 'aceras',
 'achyranthes',
 'acrostichutn',
 'adiantum',
 'aegilops',
 'aegilotricum',
 'aellenia',
 'aeluropus',
 'agropyrqn',
 'agrostemma',
 'agrostis',
 'aira',
 'airopsis',
 'aizoon',
 'alisma',
 'allium',
 'alnus',
 'aloe',
 'alopecurus',
 'alsine',
 'alternanthera',
 'amaranthus',
 'amaryllis',
 'ammochloa',
 'ammophila',
 'anabasis',
 'anacamptis',
 'androcymbium',
 'andropogon',
 'ankyropetalum',
 'anthistiria',
 'anthoxanthum',
 'antinoria',
 'apera',
 'arceuthobium',
 'arceuthos',
 'arenaria',
 'arisarum',
 'aristella',
 'aristida',
 'aristolochia',
 'arrhenatherum',
 'arthrocnemum',
 'arum',
 'arundo',
 'asarum',
 'asparagus',
 'asphodeline',
 'asphodelusi',
 'aspidium',
 'asplenium',
 'athyrium',
 'atraphaxis',
 'atriplex',
 'atropis',
 'avena',
 'axyris',
 'baldingera',
 'bassia',
 'beckmannia',
 'behen',
 'bellevalia',
 'beta',
 'biarum',
 'bilderdykia',
 'blechnum',
 'blitum',
 'blysmus',
 'boissiera',
 'bolanthus',
 'bothriochloa',
 'brachiaria',
 'b

In [325]:
len(lst)

396

In [306]:
'•˛»»'.isalpha()

False

In [307]:
genus_coord_vol1_df["word"].str.lower()

1715596    acanthophyllum
1715634            aceras
1715714       achyranthes
1715745      acrostichutn
1715778          adiantum
                ...      
1762634      zannichellia
1762657               zea
1762666           zebrina
1762687          zingeria
1762726           zostera
Name: word, Length: 419, dtype: object

In [333]:

vol_char_df = vol1_char_df 
vol_index = vol1_index

all_vol_data = [(vol1_char_df, vol1_index),
                (vol2_char_df, vol2_index),
                (vol3_char_df, vol3_index)]

for vol_char_df, vol_index in all_vol_data: 
    vol_char_df["epithet_coord_match"] = vol_char_df["span_bbox"].apply(lambda x : False)
    for page_num in tqdm(vol_index):
        margin = 1.25 * vol_char_df[(vol_char_df["epithet_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).mean()
        epithet_char_df = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["epithet_index_pat_match"] == True)]
        epithet_df = epithet_char_df.loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])].drop_duplicates()
        page_epithet_2dic = [{}, {}]
        
        for i in range(epithet_df.shape[0]):
            g_index = str(page_num) + "_" + str(i)
            p0 = epithet_df['span_bbox'].iloc[i]
            col = epithet_df['col_num'].iloc[i]
            x_ref = p0[0]

            ref_neighbors_df = epithet_df[(epithet_df["page_num"] == page_num) & 
                                        (epithet_df["span_bbox"].apply(lambda x : x_ref - margin <= x[0] and x[0] <= x_ref + margin))]
            #should say span and not have char_num to be more accurate. This only accounts for errors with span changing... 
            # ref_neighbors_df = vol_char_df[(vol_char_df["page_num"] == page_num) & 
            #                                (vol_char_df["char_num"] < 2) &
            #                                (vol_char_df["char_bbox"].apply(lambda x : x_ref - margin <= x[0] and x[0] <= x_ref + margin))]
        
            num_neighbors = ref_neighbors_df.shape[0]
            mean_neighbors = ref_neighbors_df["span_bbox"].apply(lambda x : x[0]).mean()
            page_epithet_2dic[col][g_index] = (num_neighbors, mean_neighbors)
        
        mean_left_epithet = max(page_epithet_2dic[0].values(), default = [-1, -1])[1]
        mean_right_epithet = max(page_epithet_2dic[1].values(), default = [-1, -1])[1]

        if mean_left_epithet == -1 or mean_right_epithet == -1:
            mean_valid_col = max(mean_left_epithet, mean_right_epithet)
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "epithet_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["span_bbox"].apply(lambda x : is_coord_match(x, mean_valid_col, mean_valid_col, margin))
        elif mean_left_epithet == -1 and mean_right_epithet == -1:
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "epithet_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["span_bbox"].apply(lambda x : False)
        else: 
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "epithet_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["span_bbox"].apply(lambda x : is_coord_match(x, mean_left_epithet, mean_right_epithet, margin))

100%|██████████| 28/28 [00:01<00:00, 24.97it/s]


In [335]:
#page_num = 616
# vol_char_df = vol3_char_df
# vol_index = vol3_index
# doc = vol3_doc

# image_list = []
# output_name = "strickt_match_vol3"

all_vol_data = [(vol1_char_df, vol1_index, vol1_doc, "epithet_coord_match_vol1"),
                (vol2_char_df, vol2_index, vol2_doc, "epithet_coord_match_vol2"),
                (vol3_char_df, vol3_index, vol3_doc, "epithet_coord_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data: 
    #for each volume 
    image_list = []

    max_width_genus_char = vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()
    max_width_epithet_char = vol_char_df[(vol_char_df["epithet_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        genus_db = vol_char_df[(vol_char_df['page_num'] == page_num)
                                & (vol_char_df['genus_index_pat_match'] == True) 
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['epithet_coord_match'] == True) & 
                                     (vol_char_df['word'].str.isupper() == False) & 
                                     (vol_char_df['word'].str.isnumeric() == False)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['epithet_index_pat_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #genus pattern match flag should check with half page and not entire page:
        for col in range(2):
            num_genus_col = genus_db[genus_db["col_num"] == col].shape[0]
            num_epithet_col = epithet_db[epithet_db["col_num"] == col].shape[0]
        if num_genus_col <= 2:
            print(f"***FLAG***\n number of patterns matched for genus \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
        if num_epithet_col <= 2:
            print(f"***FLAG***\n number of patterns matched for epithet \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")

        # #genus is blue, 3 
        # for coord in genus_db['span_bbox'] :
        #     x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
        #     draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)

        #epithet Coord is orange-pinkish, 5
        for coord in epithet_coord_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)

        #epithet is red, 3
        for coord in epithet_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

 65%|██████▌   | 15/23 [00:06<00:03,  2.22it/s]

***FLAG***
 number of patterns matched for genus 
 in page 15 col 1 is 2


 87%|████████▋ | 20/23 [00:09<00:01,  2.23it/s]

***FLAG***
 number of patterns matched for genus 
 in page 20 col 1 is 0


100%|██████████| 23/23 [00:10<00:00,  2.22it/s]

...

 86%|████████▌ | 24/28 [00:10<00:01,  2.40it/s]

***FLAG***
 number of patterns matched for genus 
 in page 24 col 1 is 2


 93%|█████████▎| 26/28 [00:10<00:00,  2.39it/s]

***FLAG***
 number of patterns matched for genus 
 in page 26 col 1 is 0


100%|██████████| 28/28 [00:11<00:00,  2.39it/s]


In [336]:

vol_char_df = vol1_char_df 
vol_index = vol1_index

all_vol_data = [(vol1_char_df, vol1_index),
                (vol2_char_df, vol2_index),
                (vol3_char_df, vol3_index)]

for vol_char_df, vol_index in all_vol_data: 
    vol_char_df["genus_coord_match_v2"] = vol_char_df["span_bbox"].apply(lambda x : False)
    for page_num in tqdm(vol_index):
        margin = 1.25 * vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).mean()
        genus_char_df = vol_char_df[(vol_char_df["page_num"] == page_num) &
                                    (vol_char_df["genus_index_pat_match"] == True) &
                                    (vol_char_df['epithet_coord_match'] == False)
                                   ]
        genus_df = genus_char_df.loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])].drop_duplicates()
        page_genus_2dic = [{}, {}]
        
        for i in range(genus_df.shape[0]):
            g_index = str(page_num) + "_" + str(i)
            p0 = genus_df['span_bbox'].iloc[i]
            col = genus_df['col_num'].iloc[i]
            x_ref = p0[0]

            ref_neighbors_df = genus_df[(genus_df["page_num"] == page_num) & 
                                        (genus_df["span_bbox"].apply(lambda x : x_ref - margin <= x[0] and x[0] <= x_ref + margin))]
            #should say span and not have char_num to be more accurate. This only accounts for errors with span changing... 
            # ref_neighbors_df = vol_char_df[(vol_char_df["page_num"] == page_num) & 
            #                                (vol_char_df["char_num"] < 2) &
            #                                (vol_char_df["char_bbox"].apply(lambda x : x_ref - margin <= x[0] and x[0] <= x_ref + margin))]
        
            num_neighbors = ref_neighbors_df.shape[0]
            mean_neighbors = ref_neighbors_df["span_bbox"].apply(lambda x : x[0]).mean()
            page_genus_2dic[col][g_index] = (num_neighbors, mean_neighbors)
        
        mean_left_genus = max(page_genus_2dic[0].values(), default = [-1, -1])[1]
        mean_right_genus = max(page_genus_2dic[1].values(), default = [-1, -1])[1]

        if mean_left_genus == -1 or mean_right_genus == -1:
            mean_valid_col = max(mean_left_genus, mean_right_genus)
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "genus_coord_match_v2"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["span_bbox"].apply(lambda x : is_coord_match(x, mean_valid_col, mean_valid_col, margin))
        elif mean_left_genus == -1 and mean_right_genus == -1:
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "genus_coord_match_v2"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["span_bbox"].apply(lambda x : False)
        else: 
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "genus_coord_match_v2"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["span_bbox"].apply(lambda x : is_coord_match(x, mean_left_genus, mean_right_genus, margin))

100%|██████████| 28/28 [00:00<00:00, 48.68it/s]


In [338]:
#page_num = 616
# vol_char_df = vol3_char_df
# vol_index = vol3_index
# doc = vol3_doc

# image_list = []
# output_name = "strickt_match_vol3"

all_vol_data = [(vol1_char_df, vol1_index, vol1_doc, "genusNotEpithet_coord_match_vol1"),
                (vol2_char_df, vol2_index, vol2_doc, "genusNotEpithet_coord_match_vol2"),
                (vol3_char_df, vol3_index, vol3_doc, "genusNotEpithet_coord_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data: 
    #for each volume 
    image_list = []

    max_width_genus_char = vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()
    max_width_epithet_char = vol_char_df[(vol_char_df["epithet_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        genus_db = vol_char_df[(vol_char_df['page_num'] == page_num)
                                & (vol_char_df['genus_index_pat_match'] == True) 
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        genus_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['genus_coord_match_v2'] == True) & 
                                     (vol_char_df['word'].str.isupper() == False) & 
                                     (vol_char_df['word'].str.isnumeric() == False)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['epithet_index_pat_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #genus pattern match flag should check with half page and not entire page:
        for col in range(2):
            num_genus_col = genus_db[genus_db["col_num"] == col].shape[0]
            num_epithet_col = epithet_db[epithet_db["col_num"] == col].shape[0]
        if num_genus_col <= 2:
            print(f"***FLAG***\n number of patterns matched for genus \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
        if num_epithet_col <= 2:
            print(f"***FLAG***\n number of patterns matched for epithet \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
 
        #genus Coord is orange-pinkish, 5
        for coord in genus_coord_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)

        #genus is blue, 3 
        for coord in genus_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)

        # #epithet is red, 3
        # for coord in epithet_db['span_bbox'] :
        #     x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
        #     draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#990000"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

 65%|██████▌   | 15/23 [00:06<00:03,  2.23it/s]

***FLAG***
 number of patterns matched for genus 
 in page 15 col 1 is 2


 87%|████████▋ | 20/23 [00:09<00:01,  2.24it/s]

***FLAG***
 number of patterns matched for genus 
 in page 20 col 1 is 0


100%|██████████| 23/23 [00:10<00:00,  2.23it/s]

...

 86%|████████▌ | 24/28 [00:10<00:01,  2.39it/s]

***FLAG***
 number of patterns matched for genus 
 in page 24 col 1 is 2


 93%|█████████▎| 26/28 [00:10<00:00,  2.38it/s]

***FLAG***
 number of patterns matched for genus 
 in page 26 col 1 is 0


100%|██████████| 28/28 [00:11<00:00,  2.37it/s]


In [346]:
import math

In [350]:
all_vol_data = [(vol1_char_df, vol1_index),
                (vol2_char_df, vol2_index),
                (vol3_char_df, vol3_index)]

for vol_char_df, vol_index in all_vol_data: 
    vol_char_df["infra_coord_match"] = vol_char_df["span_bbox"].apply(lambda x : False)
    for page_num in tqdm(vol_index):

        margin = 1.25 * vol_char_df[(vol_char_df["epithet_coord_match"] == True) | (vol_char_df["genus_coord_match_v2"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).mean()
        
        mean_left_epithet = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["col_num"] == 0) & (vol_char_df["epithet_coord_match"] == True)]["span_bbox"].apply(lambda x : x[0]).mean()
        mean_left_genus = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["col_num"] == 0) & (vol_char_df["genus_coord_match_v2"] == True)]["span_bbox"].apply(lambda x : x[0]).mean()
        if math.isnan(mean_left_genus):
            mean_left_genus_all = vol_char_df[(vol_char_df["col_num"] == 0) & (vol_char_df["genus_coord_match_v2"] == True)]["span_bbox"].apply(lambda x : x[0]).mean()
            mean_left_epithet_all = vol_char_df[(vol_char_df["col_num"] == 0) & (vol_char_df["epithet_coord_match"] == True)]["span_bbox"].apply(lambda x : x[0]).mean()
            mean_left_tab = mean_left_epithet_all - mean_left_genus_all
        else: 
            mean_left_tab = mean_left_epithet - mean_left_genus
        
        mean_right_epithet = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["col_num"] == 1) & (vol_char_df["epithet_coord_match"] == True)]["span_bbox"].apply(lambda x : x[0]).mean()
        mean_right_genus = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["col_num"] == 1) & (vol_char_df["genus_coord_match_v2"] == True)]["span_bbox"].apply(lambda x : x[0]).mean()
        if math.isnan(mean_right_genus):
            mean_right_genus_all = vol_char_df[(vol_char_df["col_num"] == 1) & (vol_char_df["genus_coord_match_v2"] == True)]["span_bbox"].apply(lambda x : x[0]).mean()
            mean_right_epithet_all = vol_char_df[(vol_char_df["col_num"] == 1) & (vol_char_df["epithet_coord_match"] == True)]["span_bbox"].apply(lambda x : x[0]).mean()
            mean_right_tab = mean_right_epithet_all - mean_right_genus_all
        else: 
            mean_right_tab = mean_right_epithet - mean_right_genus


        vol_char_df.loc[(vol_char_df["page_num"] == page_num) & (vol_char_df["word_num"] == 0)  , "infra_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["word_num"] == 0)]["span_bbox"].apply(lambda x : is_coord_match(x, mean_left_epithet + mean_left_tab, mean_right_epithet + mean_right_tab, margin))

100%|██████████| 28/28 [00:01<00:00, 23.50it/s]


In [351]:
#page_num = 616
# vol_char_df = vol3_char_df
# vol_index = vol3_index
# doc = vol3_doc

# image_list = []
# output_name = "strickt_match_vol3"

all_vol_data = [(vol1_char_df, vol1_index, vol1_doc, "infra_coord_match_vol1"),
                (vol2_char_df, vol2_index, vol2_doc, "infra_coord_match_vol2"),
                (vol3_char_df, vol3_index, vol3_doc, "infra_coord_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data: 
    #for each volume 
    image_list = []

    max_width_genus_char = vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()
    max_width_epithet_char = vol_char_df[(vol_char_df["epithet_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)

        infra_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['infra_coord_match'] == True) 
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()
                            
        #infra Coord is orange-pinkish, 5
        for coord in infra_coord_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)

        # #genus is blue, 3 
        # for coord in genus_db['span_bbox'] :
        #     x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
        #     draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)

        # #epithet is red, 3
        # for coord in epithet_db['span_bbox'] :
        #     x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
        #     draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#990000"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

100%|██████████| 28/28 [00:04<00:00,  6.01it/s]


In [358]:
#to see where author may be overflowing to the next line

vol_char_df[(vol_char_df['epithet_coord_match'] == True) &
            (vol_char_df['char'].str.isupper() == True) &
            (vol_char_df['word_num'] == 0) & 
            (vol_char_df['char_num'] == 0)
            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_bbox,word_num,word,genus_index_pat_match,epithet_index_pat_match,col_num,genus_coord_match,epithet_coord_match,genus_coord_match_v2,infra_coord_match
1554632,3,555,23,37,"(25.440000534057617, 451.4830322265625, 169.11...",6,0,"(1.0, 0.0)","(26.399999618530273, 509.0830078125, 114.60958...",0,...,"(26.399999618530273, 510.0010070800781, 92.314...",0,Krascheninnikovii,True,False,0,False,True,False,False
1554710,3,555,24,39,"(26.639999389648438, 528.2830200195312, 162.36...",1,0,"(1.0, 0.0)","(26.639999389648438, 537.8829956054688, 42.087...",0,...,"(26.639999389648438, 537.8829956054688, 42.087...",0,DC.,False,False,0,False,True,False,False
1565497,3,560,46,66,"(31.440000534057617, 482.0830078125, 204.32073...",1,0,"(1.0, 0.0)","(31.440000534057617, 491.9230041503906, 70.076...",0,...,"(31.440000534057617, 491.9230041503906, 70.076...",0,Wagenitz,False,False,0,False,True,False,False
1566524,3,560,93,136,"(237.1199951171875, 406.00299072265625, 408.58...",1,0,"(1.0, 0.0)","(237.36000061035156, 415.60302734375, 265.7769...",0,...,"(237.36000061035156, 415.60302734375, 265.7769...",0,Fritsch,False,False,1,False,True,False,False
1575185,3,564,87,140,"(244.8000030517578, 586.60302734375, 387.82345...",1,0,"(1.0, 0.0)","(244.8000030517578, 596.2030029296875, 268.686...",0,...,"(244.8000030517578, 596.2030029296875, 268.686...",0,Holub,True,False,1,False,True,False,False
1575488,3,565,6,13,"(31.68000030517578, 95.984375, 202.83992004394...",1,0,"(1.0, 0.0)","(32.15999984741211, 105.58441162109375, 58.434...",0,...,"(32.15999984741211, 105.58441162109375, 58.434...",0,Holub,False,False,0,False,True,False,False
1578956,3,566,73,111,"(247.44000244140625, 248.20298767089844, 418.4...",0,0,"(1.0, 0.0)","(247.44000244140625, 248.20298767089844, 309.5...",0,...,"(247.44000244140625, 249.12098693847656, 286.4...",0,Eichwaldii,True,False,1,False,True,False,False
1579044,3,566,76,116,"(246.9600067138672, 277.0030212402344, 418.339...",1,0,"(1.0, 0.0)","(247.67999267578125, 287.0830078125, 280.50738...",0,...,"(247.67999267578125, 287.0830078125, 280.50738...",0,Schrank,False,False,1,False,True,False,False
1582101,3,568,22,32,"(35.279998779296875, 254.7629852294922, 207.46...",1,0,"(1.0, 0.0)","(35.52000045776367, 264.3630065917969, 65.7565...",0,...,"(35.52000045776367, 264.3630065917969, 65.7565...",0,Kuntze,False,False,0,False,True,False,False
1583001,3,568,63,100,"(240.72000122070312, 160.68299865722656, 380.0...",1,0,"(1.0, 0.0)","(240.72000122070312, 170.2830047607422, 268.72...",0,...,"(240.72000122070312, 170.2830047607422, 268.72...",0,Kuntze,True,False,1,False,True,False,False


DIFFERENCE IN CONSECUTIVE ROWS: df["A"].diff(-1) 

CAN USE THIS TO SEE IF THERE'S A GAP ETC :]

After that:
- see diff between mean of groups. For other first words see if it has var. x. ssp. f. etc. if so create another catagory. BUT FIRST NEED TO GROUP EVERYTHING WITHING 1.5 CHAR of GENUS EPITHET AND NOT INCLUDE THEM IN THIS. 
If not any of this & same line as an epithet / genus -> AUTHOR

- then after this we can follow alphabetic + watch out for words like et etc + check upper case to fix author flowing to next line 

In [ ]:
def gmm_setup(vol_df, page_num):
    copy_df = vol_df[(vol_df['page_num'] == page_num) & 
                         (vol_df['word_num'] == 0) & 
                         (vol_df['char_num'] == 0) & (vol_df['word'].str.isalpha())
                        ].copy()
    copy_df["span_bbox_x0"] = copy_df['span_bbox'].apply(lambda x : x[0])
    copy_df["span_bbox_y0"] = copy_df['span_bbox'].apply(lambda x : x[1])
    copy_df["ones"] = np.ones(copy_df.shape[0])

    XY = copy_df[["span_bbox_x0","span_bbox_y0"]]
    X = copy_df[["span_bbox_x0","ones"]]

    n_components = np.arange(1, 15) #logically we shouldn't have more than 15 this many including some errors
    models_XY = [GMM(n, covariance_type='full', random_state=0).fit(XY) for n in n_components]
    ideal_XY_n = np.array([m.bic(XY) for m in models_XY]).argmin()+1 #BIC: Bayesian information criterion
    print("best n considering X and Y:", ideal_XY_n)
    gmm_XY = GMM(ideal_XY_n, covariance_type='full').fit(X)
    labels_XY = gmm_XY.predict(X)
    copy_df['x0_labels_XY'] = labels_XY

    models_X = [GMM(n, covariance_type='full', random_state=0).fit(X) for n in n_components]
    ideal_X_n = np.array([m.bic(X) for m in models_X]).argmin()+1 #BIC: Bayesian information criterion
    print("best n considering X only:", ideal_X_n)
    gmm_X = GMM(ideal_X_n, covariance_type='full').fit(X)
    labels_X = gmm_X.predict(X)
    copy_df['x0_labels_X'] = labels_X
    
    return copy_df

### pruning the non alphabetic characters
finding bounding box of the words

In [ ]:
vol1_pruned_df = vol1_char_df[(vol1_char_df["char"].str.isalpha())]

In [371]:
coords_str = ["char_x0", "char_y0", "char_x1", "char_y1"]
for vol_char_df in [vol1_char_df, vol2_char_df, vol3_char_df]:
    for i in range(len(coords_str)):
        vol_char_df[coords_str[i]] = vol_char_df["char_bbox"].apply(lambda x: x[i])

In [403]:
vol3_char_df.columns.difference(["char_num", "char", "char_origin",	"char_bbox", "char_x0", "char_y0", "char_x1", "char_y1"], sort=False).tolist()

['vol_num',
 'page_num',
 'block_num',
 'block_num_absolute',
 'block_bbox',
 'line_num',
 'line_wmode',
 'line_dir',
 'line_bbox',
 'span_num',
 'span_size',
 'span_flags',
 'span_font',
 'span_color',
 'span_ascender',
 'span_descender',
 'span_origin',
 'span_bbox',
 'word_num',
 'word',
 'genus_index_pat_match',
 'epithet_index_pat_match',
 'col_num',
 'genus_coord_match',
 'epithet_coord_match',
 'genus_coord_match_v2',
 'infra_coord_match']

In [ ]:
# DOESN'T WORK RIGHT !!! 

In [404]:
vol3_char_df.columns.difference(vol3_char_df.columns.difference(["char_num", "char", "char_origin",	"char_bbox", "char_x0", "char_y0", "char_x1", "char_y1"], sort=False).tolist())

Index(['char', 'char_bbox', 'char_num', 'char_origin', 'char_x0', 'char_x1',
       'char_y0', 'char_y1'],
      dtype='object')

In [426]:
#unsure group : 'genus_index_pat_match', 'epithet_index_pat_match', 'col_num', 'genus_coord_match', 'epithet_coord_match', 'genus_coord_match_v2', 'infra_coord_match'
group = vol1_char_df.columns.difference(["char_num", "char", "char_origin",	"char_bbox", "char_x0", "char_y0", "char_x1", "char_y1"], sort=False).tolist()
yee1 = vol1_char_df[vol1_char_df['char'].str.isalpha()].groupby(group).agg({'char_x0': 'min', 'char_y0': 'min', 'char_x1': 'max', 'char_y1': 'max'}).reset_index()
yee1["word_bbox"] = yee1.apply(lambda row: (row['char_x0'], row['char_y0'], row['char_x1'], row['char_y1']), axis = 1)

yee2 = vol2_char_df[vol2_char_df['char'].str.isalpha()].groupby(group).agg({'char_x0': 'min', 'char_y0': 'min', 'char_x1': 'max', 'char_y1': 'max'}).reset_index()
yee2["word_bbox"] = yee2.apply(lambda row: (row['char_x0'], row['char_y0'], row['char_x1'], row['char_y1']), axis = 1)

yee3 = vol3_char_df[vol3_char_df['char'].str.isalpha()].groupby(group).agg({'char_x0': 'min', 'char_y0': 'min', 'char_x1': 'max', 'char_y1': 'max'}).reset_index()
yee3["word_bbox"] = yee3.apply(lambda row: (row['char_x0'], row['char_y0'], row['char_x1'], row['char_y1']), axis = 1)



clearly this doesn't work the way I want it to

In [427]:
#page_num = 616
# vol_char_df = vol3_char_df
# vol_index = vol3_index
# doc = vol3_doc

# image_list = []
# output_name = "strickt_match_vol3"

all_vol_data = [(yee1, vol1_index, vol1_doc, "wordbbox_coord_match_vol1"),
                (yee2, vol2_index, vol2_doc, "wordbbox_coord_match_vol2"),
                (yee3, vol3_index, vol3_doc, "wordbbox_coord_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data: 
    #for each volume 
    image_list = []
    #apparently "char_bbox",  gone??
    #unsure group : 'genus_index_pat_match', 'epithet_index_pat_match', 'col_num', 'genus_coord_match', 'epithet_coord_match', 'genus_coord_match_v2', 'infra_coord_match'
    #unsure group : 'genus_index_pat_match', 'epithet_index_pat_match', 'col_num', 'genus_coord_match', 'epithet_coord_match', 'genus_coord_match_v2', 'infra_coord_match'
    group = vol_char_df.columns.difference(["char_num", "char", "char_origin", "char_x0", "char_y0", "char_x1", "char_y1"], sort=False).tolist()
    yee = vol_char_df.groupby(group).agg({'char_x0': 'min', 'char_y0': 'min', 'char_x1': 'max', 'char_y1': 'max'}).reset_index()
    yee["word_bbox"] = yee.apply(lambda row: (row['char_x0'], row['char_y0'], row['char_x1'], row['char_y1']), axis = 1)
    vol_char_df = yee

    #max_width_genus_char = vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()
    #max_width_epithet_char = vol_char_df[(vol_char_df["epithet_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).max()

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        genus_db = vol_char_df[(vol_char_df['page_num'] == page_num)
                                & (vol_char_df['genus_index_pat_match'] == True) 
                            ]

        genus_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['genus_coord_match_v2'] == True) & 
                                     (vol_char_df['word'].str.isupper() == False) & 
                                     (vol_char_df['word'].str.isnumeric() == False)
                            ]

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['epithet_index_pat_match'] == True)
                                ]

        #genus pattern match flag should check with half page and not entire page:
        for col in range(2):
            num_genus_col = genus_db[genus_db["col_num"] == col].shape[0]
            num_epithet_col = epithet_db[epithet_db["col_num"] == col].shape[0]
        if num_genus_col <= 2:
            print(f"***FLAG***\n number of patterns matched for genus \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
        if num_epithet_col <= 2:
            print(f"***FLAG***\n number of patterns matched for epithet \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
 
        #genus Coord is orange-pinkish, 5
        for coord in genus_coord_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)

        #genus is blue, 3 
        for coord in genus_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)

        # #epithet is red, 3
        # for coord in epithet_db['span_bbox'] :
        #     x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
        #     draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#990000"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

 65%|██████▌   | 15/23 [00:03<00:01,  4.35it/s]

***FLAG***
 number of patterns matched for genus 
 in page 15 col 1 is 2


 87%|████████▋ | 20/23 [00:04<00:00,  4.39it/s]

***FLAG***
 number of patterns matched for genus 
 in page 20 col 1 is 0


100%|██████████| 23/23 [00:05<00:00,  4.34it/s]


...

 86%|████████▌ | 24/28 [00:05<00:00,  4.69it/s]

***FLAG***
 number of patterns matched for genus 
 in page 24 col 1 is 2


 93%|█████████▎| 26/28 [00:05<00:00,  4.69it/s]

***FLAG***
 number of patterns matched for genus 
 in page 26 col 1 is 0


100%|██████████| 28/28 [00:05<00:00,  4.70it/s]
